In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gc
import lightgbm as lgb
import time
# import datetime
# import xgboost as xgb
# import time
# import itertools
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

%matplotlib inline
sns.set()

In [ ]:
# 데이터 불러오기
INPUT_DIR = '/kaggle/input/m5-forecasting-accuracy'

calendar_df = pd.read_csv(f"{INPUT_DIR}/calendar.csv")
sell_prices_df = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv")
sales_train_validation_df = pd.read_csv(f"{INPUT_DIR}/sales_train_validation.csv")
sample_submission_df = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv")

In [ ]:
calendar_df.head() # 제품 판매 날짜에 대한 정보
# date: 날짜
# wm_yr_wk:
# weekday: 요일 / # wday: 요일을 숫자로
# month: 월 / # year: 연도
# d: unique value 
# event_name_1, 2: / # event_type_1, 2:
# snap_CA: / # snap_TX: / # snap_WI: 

In [ ]:
calendar_df['event_name_1'].value_counts()

In [ ]:
calendar_df['event_type_1'].value_counts()

In [ ]:
calendar_df['event_name_2'].value_counts()

In [ ]:
calendar_df['event_type_2'].value_counts()

In [ ]:
sell_prices_df.head()  # 상점(store_id) 및 날짜(wm_yr_wk) 당 판매 된 제품(item_id) 가격(sell_price)에 대한 정보를 포함.

In [ ]:
sales_train_validation_df.head()  # 제품(item_id) 및 상점(store_id) 별 과거 일일 단위 판매 데이터를 포함
# id = {item_id}_{store_id} 로 구성됨. 

In [ ]:
sample_submission_df.head()  # 제출 파일 예시

In [ ]:
# Calendar data type cast -> Memory Usage Reduction
# Calendar 데이터 타입 변경 -> 메모리 사용 감소
calendar_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]] = calendar_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]].astype("int8")
calendar_df[["wm_yr_wk", "year"]] = calendar_df[["wm_yr_wk", "year"]].astype("int16") 
calendar_df["date"] = calendar_df["date"].astype("datetime64")

nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    calendar_df[feature].fillna('unknown', inplace = True)

calendar_df[["weekday", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] = calendar_df[["weekday", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] .astype("category")

In [ ]:
# Sales Training dataset cast -> Memory Usage Reduction
sales_train_validation_df.loc[:, "d_1":] = sales_train_validation_df.loc[:, "d_1":].astype("int16")

In [ ]:
# Make ID column to sell_price dataframe
# 다른 df 에서 쓰고 있는 id를 만들어 줌. 
sell_prices_df.loc[:, "id"] = sell_prices_df.loc[:, "item_id"] + "_" + sell_prices_df.loc[:, "store_id"] + "_validation"

In [ ]:
sell_prices_df = pd.concat([sell_prices_df, sell_prices_df["item_id"].str.split("_", expand=True)], axis=1)  # cat_id, dept_id, 뭔가를 생성함.
# column 명 바꿈
sell_prices_df = sell_prices_df.rename(columns={0:"cat_id", 1:"dept_id"})
# type 바꿈 
sell_prices_df[["store_id", "item_id", "cat_id", "dept_id"]] = sell_prices_df[["store_id","item_id", "cat_id", "dept_id"]].astype("category")
# 필요 없는 열 삭제
sell_prices_df = sell_prices_df.drop(columns=2)

In [ ]:
sell_prices_df

# 데이터 전처리

In [ ]:
# 세개의 데이터 셋을 결합한다. 
# 예측 모델에 적용하기 쉽게하기 위해 옆으로 넓은 데이터셋에서 아래로 긴 데이터 셋을 만든다.

In [ ]:
def make_dataframe():
    # Wide format dataset 
    df_wide_train = sales_train_validation_df.drop(columns=["item_id", "dept_id", "cat_id", "state_id","store_id", "id"]).T
    df_wide_train.index = calendar_df["date"][:1913]
    df_wide_train.columns = sales_train_validation_df["id"]
    
    # Making test label dataset
    df_wide_test = pd.DataFrame(np.zeros(shape=(56, len(df_wide_train.columns))), index=calendar_df.date[1913:], columns=df_wide_train.columns)
    df_wide = pd.concat([df_wide_train, df_wide_test])

    # Convert wide format to long format
    df_long = df_wide.stack().reset_index(1)
    df_long.columns = ["id", "value"]

    del df_wide_train, df_wide_test, df_wide
    gc.collect()
    
    df = pd.merge(pd.merge(df_long.reset_index(), calendar_df, on="date"), sell_prices_df, on=["id", "wm_yr_wk"])
    df = df.drop(columns=["d"])
    #     df[["cat_id", "store_id", "item_id", "id", "dept_id"]] = df[["cat_id"", store_id", "item_id", "id", "dept_id"]].astype("category")
    df["sell_price"] = df["sell_price"].astype("float16")   
    df["value"] = df["value"].astype("int32")
    df["state_id"] = df["store_id"].str[:2].astype("category")


    del df_long
    gc.collect()

    return df

df = make_dataframe()

In [ ]:
def add_date_feature(df):
    df["year"] = df["date"].dt.year.astype("int16")
    df["month"] = df["date"].dt.month.astype("int8")
    df["week"] = df["date"].dt.week.astype("int8")
    df["day"] = df["date"].dt.day.astype("int8")
    df["quarter"]  = df["date"].dt.quarter.astype("int8")
    return df

In [ ]:
df = add_date_feature(df)
df

In [ ]:
df.columns

In [ ]:
# Data Visualization
# Total Item Sold Transition


In [ ]:
temp_series = df.groupby(["cat_id", "date"])["value"].sum()
temp_series

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(temp_series[temp_series.index.get_level_values("cat_id") == "FOODS"].index.get_level_values("date"), temp_series[temp_series.index.get_level_values("cat_id") == "FOODS"].values, label="FOODS")
plt.plot(temp_series[temp_series.index.get_level_values("cat_id") == "HOUSEHOLD"].index.get_level_values("date"), temp_series[temp_series.index.get_level_values("cat_id") == "HOUSEHOLD"].values, label="HOUSEHOLD")
plt.plot(temp_series[temp_series.index.get_level_values("cat_id") == "HOBBIES"].index.get_level_values("date"), temp_series[temp_series.index.get_level_values("cat_id") == "HOBBIES"].values, label="HOBBIES")
plt.xlabel("Year")
plt.ylabel("# of sold items")
plt.title("Total Item Sold Transition of each Category")
plt.legend()

Point of the graph

FOODS is the most sold item category of these three categories.
HOUSEHOLD is the 2nd one, and HOBBIES are the least sold one.
FOODS category appearently has some periodical feature.
During one year, it seems more items are sold in summer than in winter, however, we have to verify this.
As for more short time interval, it seems the trend has monthly or weekly features. (Let's take a look below)

HOUSEHOLD category items sold is gradually increasing from 2011.
However, it may be because some items are not in the store in 2011.
So we have to take the total item in the store into account. Periodical Features are not so clear in this category compared to FOODS.

In HOBBIES category, periodical features are less appearent like HOUSEHOLD category.

In some point (around the end of year), all categories don't have any sold. So I think we have to consider whether we take these days into account when training models.

So let's take a look at the latest year, 2015!

In [ ]:
temp_series = temp_series.loc[temp_series.index.get_level_values("date") >= "2015-01-01"]
plt.figure(figsize=(12, 4))
plt.plot(temp_series[temp_series.index.get_level_values("cat_id") == "FOODS"].index.get_level_values("date"), temp_series[temp_series.index.get_level_values("cat_id") == "FOODS"].values, label="FOODS")
plt.plot(temp_series[temp_series.index.get_level_values("cat_id") == "HOUSEHOLD"].index.get_level_values("date"), temp_series[temp_series.index.get_level_values("cat_id") == "HOUSEHOLD"].values, label="HOUSEHOLD")
plt.plot(temp_series[temp_series.index.get_level_values("cat_id") == "HOBBIES"].index.get_level_values("date"), temp_series[temp_series.index.get_level_values("cat_id") == "HOBBIES"].values, label="HOBBIES")
plt.xlabel("Year-Month")
plt.ylabel("# of sold items")
plt.title("Total Item Sold Transition of each Category from 2015")
plt.legend()